# Get a Biomass function using a GSM
**Model Publication:**  
Kugler, A., Stensjö, K. Optimal energy and redox metabolism in the cyanobacterium Synechocystis sp. PCC 6803. npj Syst Biol Appl 9, 47 (2023). https://doi.org/10.1038/s41540-023-00307-3

**Available at:**  
https://github.com/amitkugler/CBA

In [67]:
import pandas as pd

from cobra.io import load_json_model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba

from cobra import Model, Reaction, Metabolite

In [2]:
# Load the model
model = load_json_model('../CBA/GSM/JSON/iJN678_AK.json')

In [3]:
def add_import(model, metabolite_name, reaction_stoichiometry, bounds=(-1000,1000)):
    # Define the reaction
    reac = Reaction(f'{metabolite_name}_import')
    reac.name = f'{metabolite_name} import'
    reac.bounds = bounds

    # Add the correct stoichiometry and add to the model
    reac.add_metabolites({model.metabolites.get_by_id(k): v for k,v in reaction_stoichiometry.items()})
    model.add_reactions([reac])
    
    return model

In [18]:
with model:
    #############
    # Set phototrophic growth and modify the model as described in the Analysis by Kugler

    # #set model objective to autotrophic growth
    # model.objective = 'BIOMASS_Ec_SynAuto'
    # #set photoautotropic growth by constraining glucose exchange reatcion
    # model.reactions.EX_glc__D_e.bounds = (0, 1000)
    # #set photoautotropic growth by constraining HCO3 exchange reatcion.
    # model.reactions.EX_co2_e.bounds = (0, 1000)
    # model.reactions.EX_hco3_e.bounds = (-3.7, 1000)
    # #set photon flux to lab scale conditions.
    # model.reactions.EX_photon_e.bounds = (-45, -45)
    # #blocking transhydrogenase PntAB
    # model.reactions.NADTRHD.bounds = (0, 0)

    test = loopless_solution(model)

In [75]:
def get_stoichiometric_fluxes(model, solution, compound):
    # Get the reactions where the compound is involved and their stoichiometry
    react = solution.fluxes[[x.id for x in model.metabolites.get_by_id(compound).reactions]]
    stoich = pd.Series({x:model.reactions.get_by_id(x).metabolites[model.metabolites.get_by_id(compound)] for x in react.index})
    
    # Return the scaled flux according to the stoichiometry
    return react * stoich

In [314]:
solutions = {}

with model:
    #############
    # Set phototrophic growth and modify the model as described in the Analysis by Kugler

    #set model objective to autotrophic growth
    model.objective = 'BIOMASS_Ec_SynAuto'
    #set photoautotropic growth by constraining glucose exchange reatcion
    model.reactions.EX_glc__D_e.bounds = (0, 1000)
    #set photoautotropic growth by constraining HCO3 exchange reatcion.
    model.reactions.EX_co2_e.bounds = (0, 1000)
    model.reactions.EX_hco3_e.bounds = (-3.7, 1000)
    #set photon flux to lab scale conditions.
    model.reactions.EX_photon_e.bounds = (-45, -45)
    #blocking transhydrogenase PntAB
    model.reactions.NADTRHD.bounds = (0, 0)

    # Make the model loopless (makes model infeasible)
    # add_loopless(model)

    # Get the default flux
    solutions["default"] = model.optimize("maximize")
    solutions["default_pfba"] = pfba(model)

    ##############
    # With imports
    # Disable light reactions
    model.reactions.EX_photon_e.bounds = (0, 0)

    # Disable NDH-1 as we want to measure ATP import
    # model.reactions.get_by_id("NDH1_1u").bounds = (0, 0)
    # model.reactions.get_by_id('NDH1_2u').bounds = (0, 0)
    # model.reactions.get_by_id('NDH1_3u').bounds = (0, 0)
    # model.reactions.get_by_id('NDH1_4pp').bounds = (0, 0)
    

    # # Disable Terminal oxidases
    model.reactions.get_by_id('CYO1b_syn').bounds = (0, 0) # COX
    model.reactions.get_by_id("CYTBDu").bounds = (0, 0) # Cyd
    model.reactions.get_by_id("CYTBDpp_1").bounds = (0, 0) # Cyd
    model.reactions.get_by_id('Flv2_4').bounds = (0, 0) # Flv2/4
    model.reactions.get_by_id('ARTO').bounds = (0, 0) # ARTO

    # # Disable H2 production
    # model.reactions.get_by_id("H2ASE_syn").bounds = (0, 0)

    # # Disable Cyt b6f
    # model.reactions.get_by_id('CBFC2').bounds = (0, 0)

    # # Disable the Mehler reaction
    model.reactions.get_by_id("MEHLER").bounds = (0, 0)

    # # model.reactions.get_by_id("ATPS4rpp_1").bounds = (0,0)
    # model.reactions.get_by_id("ATPSu").bounds = (0,0)

    # Instead, add Exchange reactions for ATP, NADPH and 3PGA
    pseudo_reactions = {
        "3PGA":  ({"3pg_c":1}, (-1, 1)),
        "ATP":   ({"atp_c":1, "adp_c":-1}, (0, 1000)),
        "NADPH": ({"nadph_c":1, "h_c":1, "nadp_c":-1}, (0, 1000)),
        "H+": ({"h_c":1}, (-1000, 1000)),
    }
    for name, (stoich, bounds) in pseudo_reactions.items():
        model = add_import(model, name, stoich, bounds)

    # Set a new objective function
    model.objective = {
        model.reactions.get_by_id('BIOMASS_Ec_SynAuto'):0.9,
        model.reactions.get_by_id('ATP_import'):0.1,
    
    }

    # Disable other Carbon import reactions
    # HCO3 import
    # model.reactions.EX_hco3_e.lower_bound = 0

    solutions["imports"] = model.optimize("maximize")
    solutions["imports_pfba"] = pfba(model)

    print(model.summary(solutions["imports"]))

# Print a summary of the models
for nam, sol in solutions.items():
    print(f"""
model: {nam}
biomass: {sol.fluxes.get("BIOMASS_Ec_SynAuto")}
ATP: {sol.fluxes.get("ATP_import")}
NADPH: {sol.fluxes.get("NADPH_import")}
3PGA: {sol.fluxes.get("3PGA_import")}
""")

Objective
0.9 BIOMASS_Ec_SynAuto + 0.1 ATP_import = 0.11619772353098551

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e 7.542E-05         0   0.00%
 cobalt2_e EX_cobalt2_e 5.436E-05         0   0.00%
     cu2_e     EX_cu2_e 5.028E-05         0   0.00%
     fe2_e     EX_fe2_e 0.0001251         0   0.00%
     fe3_e     EX_fe3_e 0.0001139         0   0.00%
       h_e       EX_h_e     51.67         0   0.00%
    hco3_e    EX_hco3_e       3.7         1 100.00%
       k_e       EX_k_e  0.002828         0   0.00%
     mg2_e     EX_mg2_e 0.0004808         0   0.00%
     mn2_e     EX_mn2_e 5.063E-05         0   0.00%
    mobd_e    EX_mobd_e 5.064E-05         0   0.00%
     na1_e     EX_na1_e  6.33E-05         0   0.00%
     no3_e     EX_no3_e    0.1507         0   0.00%
      o2_e      EX_o2_e  0.009053         0   0.00%
     so4_e     EX_so4_e  0.003171         0   0.00%
     zn2_e     EX_zn2_e 5.028E-05         0   0.00%
  rdmbzi_c  SK_rdmbzi_c 3.728

In [311]:
comp = "atp_c"
flux = get_stoichiometric_fluxes(model, solutions["imports_pfba"], comp)
flux.sort_values()

PGK                   -6.292459
PRUK                  -3.709389
BIOMASS_Ec_SynAuto    -0.894048
NO3abcpp              -0.150675
ACCOAC                -0.049223
                        ...    
CYSTRS                -0.000000
PPK2                   0.000004
URIDK2r                0.000441
PYK                    0.012969
ATPS4rpp_1            10.278579
Length: 157, dtype: float64

In [292]:
model.reactions.get_by_id(flux.idxmax())

Reaction identifier,ICDHyr
Name,Isocitrate dehydrogenase (NADP)
Memory address,0x73e5a673bd60
Stoichiometry,icit_c + nadp_c --> akg_c + co2_c + nadph_c Isocitrate + Nicotinamide adenine dinucleotide phosphate --> 2-Oxoglutarate + CO2 CO2 + Nicotinamide adenine dinucleotide phosphate - reduced
GPR,icd
Lower bound,0.0
Upper bound,1000.0


In [293]:
model.reactions.get_by_id(flux.idxmin())

Reaction identifier,NDH1_4pp
Name,Active co2 transporter facilitator (periplasm)
Memory address,0x73e5a6043ac0
Stoichiometry,co2_p + h2o_c + 3.0 h_c + nadph_c + pq_p --> 3.0 h_p + hco3_c + nadp_c + pqh2_p CO2 CO2 + H2O H2O + 3.0 H+ + Nicotinamide adenine dinucleotide phosphate - reduced + Plastoquinone --> 3.0 H+ + Bicarbonate + Nicotinamide adenine dinucleotide phosphate + Plastoquinol
GPR,ndhA and ndhB and ndhC and ndhE and ndhG and ndhH and ndhI and ndhJ and (ndhK or sll8031) and...
Lower bound,0.0
Upper bound,1000.0


In [222]:
solutions["imports"]

,fluxes,reduced_costs
EX_ac_e,0.780510,1.135629e-16
34DHOXPEGOX,0.000000,-0.000000e+00
EX_photon_e,0.000000,0.000000e+00
34HPPOR,0.000007,0.000000e+00
EX_ca2_e,-0.000062,0.000000e+00
...,...,...
Flv2_4,0.000000,8.296147e-03
NGAM,0.000000,-1.935768e-02
3PGA_import,1.000000,2.765382e-02
ATP_import,-0.990456,-0.000000e+00


In [223]:
model.reactions.get_by_id(x).metabolites[model.metabolites.get_by_id("atp_c")]

-1.0

In [51]:
dir(model.metabolites.get_by_id("atp_c"))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_bound',
 '_id',
 '_model',
 '_reaction',
 '_repr_html_',
 '_set_id_with_model',
 'annotation',
 'charge',
 'compartment',
 'constraint',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_model',
 'shadow_price',
 'summary',
 'y']

In [48]:
model.reac

Reaction identifier,GLU5K
Name,Glutamate 5-kinase
Memory address,0x73e5a685e020
Stoichiometry,atp_c + glu__L_c --> adp_c + glu5p_c ATP C10H12N5O13P3 + L-Glutamate --> ADP C10H12N5O10P2 + L-Glutamate 5-phosphate
GPR,proB
Lower bound,0.0
Upper bound,1000.0


In [37]:
sol.fluxes[[x.id for x in model.metabolites.get_by_id("atp_c").reactions]]

GLU5K                   0.016914
PRAIS                   0.023215
NGAM                    0.000000
ACGK                    0.019483
HEX7                    0.000000
                          ...   
PRASCSi                 0.023215
HSK                     0.038820
TYRTRS                  0.000000
BIOMASS_Ec_SynHetero    0.000000
NADS2                   0.000163
Name: fluxes, Length: 158, dtype: float64

In [38]:
dir(model.reactions)

['34DHOXPEGOX',
 '34HPPOR',
 '3HAD100',
 '3HAD120',
 '3HAD121',
 '3HAD140',
 '3HAD141',
 '3HAD160',
 '3HAD161',
 '3HAD180',
 '3HAD181',
 '3HAD40',
 '3HAD60',
 '3HAD80',
 '3OAR100',
 '3OAR120',
 '3OAR121',
 '3OAR140',
 '3OAR141',
 '3OAR160',
 '3OAR161',
 '3OAR180',
 '3OAR181',
 '3OAR40',
 '3OAR60',
 '3OAR80',
 '3OAS100',
 '3OAS120',
 '3OAS121',
 '3OAS140',
 '3OAS141',
 '3OAS160',
 '3OAS161',
 '3OAS180',
 '3OAS181',
 '3OAS60',
 '3OAS80',
 'AACOAR_syn',
 'ABTA',
 'ABUTD',
 'ACACT1r',
 'ACBIPGT',
 'ACCOAC',
 'ACGK',
 'ACGS',
 'ACHBS',
 'ACKr',
 'ACLS',
 'ACOATA',
 'ACONT',
 'ACOTA',
 'ACP1_FMN',
 'ACS',
 'ACt2rpp',
 'ACtex',
 'ADCL',
 'ADCPS1',
 'ADCPS2',
 'ADCS',
 'ADCYRS',
 'ADK1',
 'ADMDC',
 'ADNCYC',
 'ADOCBLS',
 'ADPRDP',
 'ADPT',
 'ADPT2',
 'ADSK',
 'ADSL1r',
 'ADSL2r',
 'ADSS',
 'AGMT',
 'AGPAT160',
 'AGPAT161',
 'AGPAT180',
 'AGPAT181',
 'AGPAT181_9',
 'AGPAT182_9_12',
 'AGPAT183_6_9_12',
 'AGPAT183_9_12_15',
 'AGPAT184_6_9_12_15',
 'AGPR',
 'AGTi',
 'AHCi',
 'AHMMPS',
 'AHSERL2',


In [30]:
solutions["default_pfba"].fluxes['BIOMASS_Ec_SynAuto']

0.08239889629798884

In [27]:
solutions["default_pfba"]

,fluxes,reduced_costs
EX_ac_e,0.000000,461.333333
34DHOXPEGOX,0.000000,2.000000
EX_photon_e,-45.000000,17.000000
34HPPOR,0.000043,-2.000000
EX_ca2_e,-0.000372,2.000000
...,...,...
GLYCLc,0.008998,-2.000000
NDH2_1p,0.000000,48.000000
ARTO,0.000000,-2.000000
Flv2_4,0.000000,28.000000


In [5]:
sol.fluxes.get("ATP_import")
sol.fluxes.get("NADPH_import")
sol.fluxes.get("3PGA_import")

-0.8895669489038693